# Title

Project website: https://docs.google.com/document/d/1GkK28wOyjTPZEZuOumKPU0z1NzVT_9sxETjPOcLPVYo/edit?tab=t.0#heading=h.qifoo7co6qtd

Professor website: https://malchiodi.di.unimi.it/teaching/AMD-DSE/2024-25/en

In [1]:
!git clone https://github.com/chiesastefano/massiveData
import os
os.chdir("massiveData")

Cloning into 'massiveData'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 101 (delta 49), reused 55 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 111.92 KiB | 842.00 KiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
import subprocess
import sys
# Install required packages from the requirement.txt file if not already installed
!pip install -r requirements.txt

In [3]:
import os
import subprocess
import sys

# Set environment variables for Kaggle API
os.environ['KAGGLE_USERNAME'] = "ilchurch"
os.environ['KAGGLE_KEY'] = "5449548f4b6c8556c8f48f4a31b4ea6e"

# Ensure kaggle is installed
try:
    import kaggle
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kaggle"])
    import kaggle

# Import and authenticate using KaggleApi
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Check if the dataset file already exists
if not os.path.exists('data/Books_rating.csv'):
    api.dataset_download_files('mohamedbakhet/amazon-books-reviews', path='data', unzip=True)
else:
    print("Dataset already exists.")

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews


In [60]:
# Load the dataset with spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("JaccardSimilarity") \
    .master("local[*]") \
    .getOrCreate()
books_rating = spark.read.csv("data/Books_rating.csv", header=True, inferSchema=True)

In [61]:
# Display the first few rows of the books_rating dataset
books_rating.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

# Create a subsample of the dataset for performance reasons
I will use a sample of the dataset as a test case. My local and Google Collab computers are not powerful enough to handle the full dataset. I will use *sample* as a parameter can it can be changed to the desired sample size.

In [62]:
sample = 0.01 # 1% of the dataset

In [63]:
books_rating_sample = books_rating.sample(fraction=sample, seed=42)
books_rating_sample.show(5)
sample_size = books_rating_sample.count()

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0312322291|King James: Belie...| NULL|          NULL|                NULL|               2/2|         5.0| 1118188800|Yo Homie this is ...|"LeBron James is ...|
|B0006D6DRK|Open marriage;: A...| NULL|A3KBF2S2MGN48O|"T. Thompson ""&#...|               2/2|         4.0| 1188604800|A Must Read for a...|This book is a cl...|
|0671551345|Night World: Daug...| NULL|          NULL|                NULL|               0/0|         5.0|  889920000|The most charming...|The plot and char...|
|B0008852GG|The soul of man 

In [64]:
sample_size

30332

# Checking for null values in the 'review/text' column
Since we are using text data, we need to check for null values in the 'review/text' column in order to take a decision on how to handle them.

In [65]:
# Check for null values in the 'review/text' column
null_rows_df = books_rating.filter(
    books_rating['review/text'].isNull() | (books_rating['review/text'].rlike('^\\s*$'))
)

null_rows_df.show(30)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|        Id|               Title|               Price|             User_id|         profileName|  review/helpfulness|        review/score|         review/time|      review/summary|review/text|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|B00085T7O2|"""Catch 'em aliv...|",,ASEH0CVYVGZCZ,...|                 0/1|                 5.0|          1341705600|Fascinating life ...|There are many id...|                NULL|       NULL|
|B00085T7O2|"""Catch 'em aliv...|",,A91QQ7IYBAK2Q,...| terrible forward...|I found the book ...|                NULL|                NULL|                NULL|                NULL|       NULL|
|B0007DMFZS|"""Beloved friend...|",

Since the 'review/text' null values are more or less duplicates.

They share the same columns, besides the identifiers and the Title. Although the Title is not the same, with a qualitative check we can see they are about the same book.

In [66]:
from pyspark.sql.functions import col
filtered_df = books_rating.filter(
    col("Title").contains("Lord of the Rings")
).select("Title", "review/text").limit(100)

# Show the result
filtered_df.show(100, truncate=False)

+---------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

There are many others Lord of The Rings reviews. I think we can't discard the NA values and it's better to keep them. We can use also the Title as similarity check.

In [67]:
# Check for null values in the 'review/text' column
null_rows_df_sample = books_rating_sample.filter(
    books_rating_sample['review/text'].isNull() | (books_rating_sample['review/text'].rlike('^\\s*$'))
)

null_rows_df_sample.show(30)

+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|        Id|               Title|Price|User_id|profileName|review/helpfulness|review/score|review/time|      review/summary|review/text|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+
|B000I1VJLA|The Lord of the R...| NULL|   NULL|       NULL|               0/1|         5.0|  938563200|have only one wor...|       NULL|
+----------+--------------------+-----+-------+-----------+------------------+------------+-----------+--------------------+-----------+



The is a null value in the subsample

# Working with the subsample

## Jaccard Similarity Approach
Since the dataset is too big, I will use a subsample of the dataset to test the Jaccard Similarity approach, for test purposes. I will then try top scale it to the full dataset.
In addition, I will use the MapReduce approach to parallelize the computation of the Jaccard Similarity.

In [68]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType

In [69]:
tokenizer = RegexTokenizer(inputCol="review/text", outputCol="reviews/tokens", pattern="\\W") # \\W is a regex pattern that matches any non-word character

It tokenizes the text into words. I am trying this approach because I believe it will be faster then some neural network based approach, since this use sparks. It will have some limitations, since it does not take in account of compound words, but I think it will be good for the first approach.

In [70]:
stopwords_remover = StopWordsRemover(inputCol="reviews/tokens", outputCol="tokens/nostopwords")

StopWordsRemover() uses a predefined list of stop words in English. It removes common words that do not carry much meaning, such as "the", "is", "in", etc. This is important, since the Jaccard Similarity is based on the number of common words between two texts. If we don't remove the stop words, the Jaccard Similarity will be biased by the number of stop words in the texts, which are not relevant for the meaning of the texts.

In [71]:
def remove_numbers(tokens):
    return [token for token in tokens if not token.isdigit()]

Numbers are often not relevant for the meaning of a review. In our dataset, the score is stored in a separated column, so we can remove the numbers from the text.

In [72]:
remove_numbers_udf = udf(remove_numbers, ArrayType(StringType())) # Specify the return type as ArrayType(StringType), meaning an array/list of strings

Sparks does not work as a Python object, so we need UDF (user defined function) to use it in the pipeline.

In [73]:
tokenized_books_rating_sample = tokenizer.transform(books_rating_sample) # Apply the tokenizer to the dataset
tokenized_reviews_nostopwords_sample = stopwords_remover.transform(tokenized_books_rating_sample) # Remove the stop words from the tokenized dataset
tokenized_rw_nsw_nn_sample = tokenized_reviews_nostopwords_sample.withColumn("tokens/nostopwords/nonumbers", remove_numbers_udf(col("tokens/nostopwords"))) # Remove the numbers from the dataset

In [74]:
import sys
print("Python Version:", sys.version)  # Check if it prints Python 3.12 in both worker and driver.

Python Version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]


In [75]:
# Verify that the environment variable is set correctly

books_rating_sample.show(10)
tokenized_rw_nsw_nn_sample.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0312322291|King James: Belie...| NULL|          NULL|                NULL|               2/2|         5.0| 1118188800|Yo Homie this is ...|"LeBron James is ...|
|B0006D6DRK|Open marriage;: A...| NULL|A3KBF2S2MGN48O|"T. Thompson ""&#...|               2/2|         4.0| 1188604800|A Must Read for a...|This book is a cl...|
|0671551345|Night World: Daug...| NULL|          NULL|                NULL|               0/0|         5.0|  889920000|The most charming...|The plot and char...|
|B0008852GG|The soul of man 

In [76]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col

df_safe = tokenized_rw_nsw_nn_sample \
    .filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens/nostopwords/nonumbers",  # No renaming here
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model = cv.fit(df_safe)
vocab = model.vocabulary


In [77]:
top30 = model.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top30, start=1):
    print(f"{i:2d}. {token}")

Top 30 tokens (approx. by frequency):
 1. book
 2. read
 3. one
 4. quot
 5. story
 6. like
 7. books
 8. time
 9. good
10. great
11. reading
12. life
13. first
14. well
15. many
16. much
17. love
18. people
19. also
20. really
21. characters
22. novel
23. even
24. get
25. way
26. author
27. world
28. think
29. written
30. years


This method approximates the top 30 words in the vocaboulary. However, it doesn't show the frequency, since CountVectorizer() only approximates the real frequency of the top words. I want to find them.

In [78]:
from pyspark.sql.functions import explode, col

# 1) Explode the original tokens column into one token per row
exploded = df_safe.select(explode(col("tokens/nostopwords/nonumbers")).alias("token"))

# 2) Filter to only the top-K tokens
topK_set = set(model.vocabulary[:20])  # assumes you want top 20 from vocab
filtered = exploded.filter(col("token").isin(topK_set))

# 3) Group and count
exact_counts = (
    filtered
      .groupBy("token")
      .count()
      .orderBy(col("count").desc())
)

exact_counts.show(truncate=False)

+-------+-----+
|token  |count|
+-------+-----+
|book   |48709|
|read   |20343|
|one    |14639|
|quot   |13450|
|story  |10174|
|like   |9026 |
|books  |7476 |
|time   |7341 |
|good   |7216 |
|great  |7193 |
|reading|6900 |
|life   |6792 |
|first  |6398 |
|well   |6362 |
|many   |6135 |
|much   |5588 |
|love   |5393 |
|people |5332 |
|also   |5265 |
|really |5264 |
+-------+-----+



In [79]:
threshold = 0.20 * sample_size
print(threshold)

6066.400000000001


I defined a function that only keeps the tokens which are not in the custom_stopwords list

In [80]:
high_freq_tokens = (
    exact_counts
      .filter(col("count") > threshold)
      .select("token")
      .rdd.flatMap(lambda x: x)
      .collect()
)

In [81]:
high_freq_tokens

['book',
 'read',
 'one',
 'quot',
 'story',
 'like',
 'books',
 'time',
 'good',
 'great',
 'reading',
 'life',
 'first',
 'well',
 'many']

In [85]:
# Define a UDF to filter them out
def remove_custom_stopwords(tokens):
    return [t for t in tokens if t not in high_freq_tokens]

remove_udf = udf(remove_custom_stopwords, ArrayType(StringType()))

tokenized_sample_2 = tokenized_rw_nsw_nn_sample.withColumn(
    "tokens_clean",
    remove_udf(col("tokens/nostopwords/nonumbers"))
)

In [88]:
df_safe = tokenized_sample_2 \
    .filter(col("review/text").isNotNull())

cv = CountVectorizer(
    inputCol="tokens_clean",  # No renaming here
    outputCol="features",
    vocabSize=5000,
    minDF=10
)

model2 = cv.fit(df_safe)
vocab2 = model2.vocabulary


In [89]:
top30 = model2.vocabulary[:30]
print("Top 30 tokens (approx. by frequency):")
for i, token in enumerate(top30, start=1):
    print(f"{i:2d}. {token}")

Top 30 tokens (approx. by frequency):
 1. much
 2. love
 3. people
 4. also
 5. really
 6. characters
 7. novel
 8. even
 9. get
10. way
11. author
12. world
13. think
14. written
15. years
16. know
17. best
18. work
19. new
20. find
21. little
22. found
23. never
24. make
25. two
26. man
27. see
28. ve
29. recommend
30. reader
